# Initializing necessary Packages

In [1]:
from data_pipeline_utils import Database

import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib  inline
import re


import warnings
warnings.filterwarnings("ignore")

# Data Loading 

## Redshift

In [2]:
# Initialize database objects
mdm = Database('mdm')
awesomo = Database('awesomo')

In [3]:
# MIA Devices Attributes
df_mia_redshift = awesomo.to_dataframe(query='''SELECT * FROM SUNNAVA.MIA_DEVICES_VAR_V1 ''')
print("Shape: ",df_mia_redshift.shape)

print("Number of unique assets: ",df_mia_redshift.asset_name.nunique())

# Filtering to Wallboards
df_mia_redshift_wb = df_mia_redshift[df_mia_redshift.device_type == 'WALLBOARD']
print("------- Wallboards -------")
print("Shape: ",df_mia_redshift_wb.shape)

print("Number of unique wallboard assets: ",df_mia_redshift_wb.asset_name.nunique())

Shape:  (43585, 21)
Number of unique assets:  36156
------- Wallboards -------
Shape:  (13916, 21)
Number of unique wallboard assets:  10308


In [5]:
# NON MIA Devices Attributes
df_nonmia_redshift = awesomo.to_dataframe(query='''SELECT * FROM SUNNAVA.NONMIA_DEVICES_VAR_V1 ''')
print("Shape: ",df_nonmia_redshift.shape)

print("Number of unique assets: ",df_nonmia_redshift.asset_tag.nunique())

# Filtering to Wallboards
df_nonmia_redshift_wb = df_nonmia_redshift[df_nonmia_redshift.device_type == 'WALLBOARD']
df_nonmia_redshift_wb.rename(columns={'asset_tag':'asset_name'},inplace = True)
print("------- Wallboards -------")
print("Shape: ",df_nonmia_redshift_wb.shape)

print("Number of unique wallboard assets: ",df_nonmia_redshift_wb.asset_name.nunique())

Shape:  (2077251, 22)
Number of unique assets:  45207
------- Wallboards -------
Shape:  (1007953, 22)
Number of unique wallboard assets:  19958


# Data Preparation

In [8]:
# Creating Asset_ids and MIA/NON-MIA devices

#concating asset ids of both MIA and Non MIA devices along with the respective flags
df_final = pd.concat([pd.DataFrame({'asset_id':df_mia_redshift_wb.asset_name.unique(),
                                   'y_flag': pd.Series([1 for x in range(df_mia_redshift_wb.asset_name.nunique())],
                                                       )}),
                      pd.DataFrame({'asset_id':df_nonmia_redshift_wb.asset_name.unique(),
                                   'y_flag': pd.Series([0 for x in range(df_nonmia_redshift_wb.asset_name.nunique())],
                                                      )})],
                    axis = 0)

# Converting flag to categorical datatype
df_final['y_flag']= df_final.y_flag.astype('category')

print(df_final.shape)
print("")
print("Distribution of Test & Control:")
print("")
print(df_final.groupby('y_flag')['asset_id'].count())

(30266, 2)

Distribution of Test & Control:

y_flag
0    19958
1    10308
Name: asset_id, dtype: int64


In [9]:
df_mia_redshift_wb.columns

Index(['asset_name', 'device_type', 'software_version', 'free_disk_space',
       'is_battery_charging', 'battery_charge_level', 'min_free_space_by_type',
       'manufacturer', 'model', 'sku', 'processor', 'screen_size', 'avg_times',
       'avg_days', 'state', 'facility_type', 'ranking', 'account_type',
       'specialty', 'business_tier', 'clinic_had_mia'],
      dtype='object')

## One Hot encoding

In [53]:
df_mia_redshift_wb_2 = df_mia_redshift_wb.copy()

df_nonmia_redshift_wb_2 = df_nonmia_redshift_wb.copy()

In [54]:
df_mia_redshift_wb.replace(np.nan,'unknown',inplace=True)
df_nonmia_redshift_wb.replace(np.nan,'unknown',inplace=True)

In [84]:
## Replacing unknowns such as nil, empty, APK not installed to Unknown
df_mia_redshift_wb['software_version'] = np.where((df_mia_redshift_wb['software_version'] == '[APK NOT INSTALLED]') | 
                                                  (df_mia_redshift_wb['software_version'] == 'nil') |
                                                  (df_mia_redshift_wb['software_version'] == 'None'),
                                                  'unknown',
                                                  df_mia_redshift_wb['software_version'])

df_mia_redshift_wb['facility_type'] = np.where((df_mia_redshift_wb['facility_type'] == 'Other'),
                                               'unknown',
                                               df_mia_redshift_wb['facility_type'])

df_nonmia_redshift_wb['software_version'] = np.where((df_nonmia_redshift_wb['software_version'] == '[APK NOT INSTALLED]') | 
                                                  (df_nonmia_redshift_wb['software_version'] == 'nil') |
                                                  (df_nonmia_redshift_wb['software_version'] == 'NOT INSTALLED') |
                                                  (df_nonmia_redshift_wb['software_version'] == 'None'),
                                                  'unknown',
                                                  df_nonmia_redshift_wb['software_version'])

df_nonmia_redshift_wb['facility_type'] = np.where((df_nonmia_redshift_wb['facility_type'] == 'Other'),
                                               'unknown',
                                               df_nonmia_redshift_wb['facility_type'])

In [91]:
def one_hotencoding(data,var_list):
    ''' Function for one-hot endcoding categorical variables  '''
    
    df_categorical = pd.DataFrame()
    for column in var_list:
        curr_cols = ['asset_name']
        curr_cols.append(column)
        
        df_curr = data[curr_cols]
        # dropping duplicates
        df_curr.drop_duplicates(inplace = True)
        # Get one hot encoding of current column 
        one_hot = pd.get_dummies(df_curr[column])
        # Drop column B as it is now encoded
        df_curr = df_curr.drop(column,axis = 1)
        # Join the encoded df
        df_curr = df_curr.join(one_hot)
        df_curr = df_curr.groupby('asset_name').sum()  
        df_curr.reset_index(inplace=True)
        # dropping a column to avoid dummy variable trap
        df_curr = df_curr.iloc[:,:df_curr.shape[1]-1]
        
        # Converting all 
        cols=[i for i in df_curr.columns if i not in ["asset_name"]]
        for col in cols:
            df_curr[col]= df_curr[col].astype('category')
        if df_categorical.empty :
            df_categorical = df_curr
        else:
            df_categorical = df_categorical.merge(df_curr, left_on = 'asset_name',right_on = 'asset_name')
     
    return df_categorical

In [92]:
categorical_vars = ['software_version','manufacturer','model','sku','state','is_battery_charging',
                    'facility_type','ranking','specialty']


In [96]:
df_mia_redshift_wb_cat = one_hotencoding(df_mia_redshift_wb,categorical_vars)
df_mia_redshift_wb_cat.shape

(10308, 156)

In [97]:
df_nonmia_redshift_wb_cat = one_hotencoding(df_nonmia_redshift_wb,categorical_vars)
df_nonmia_redshift_wb_cat.shape

(19958, 157)

In [100]:
# Dropping these columns as only one of the cohorts has data 
columns_to_drop = list(set(df_nonmia_redshift_wb_cat.columns)-set(df_mia_redshift_wb_cat.columns)) + list(set(df_mia_redshift_wb_cat.columns)-set(df_nonmia_redshift_wb_cat.columns))

In [102]:
for col in columns_to_drop:
    if col in df_mia_redshift_wb_cat.columns:
        df_mia_redshift_wb_cat.drop(col,1,inplace = True)
    if col in df_nonmia_redshift_wb_cat.columns:    
        df_nonmia_redshift_wb_cat.drop(col,1,inplace = True)

print("Shape of MIA Devices :",df_mia_redshift_wb_cat.shape)    
print("Shape of NONMIA Devices :",df_nonmia_redshift_wb_cat.shape)    

Shape of MIA Devices : (10308, 150)
Shape of NONMIA Devices : (19958, 150)


# Rough

In [89]:
df_curr[df_curr.asset_name == 'W01E120225'].sum(axis = 1)

2825    4
dtype: int64

In [ ]:
# df_mia_redshift_wb['manufacturer'] = np.where((df_mia_redshift_wb['manufacturer'] == 'None'),
#                                                'unknown',
#                                                df_mia_redshift_wb['manufacturer'])

# df_mia_redshift_wb['model'] = np.where((df_mia_redshift_wb['model'] == 'None'),
#                                                'unknown',
#                                                df_mia_redshift_wb['model'])

# df_mia_redshift_wb['sku'] = np.where((df_mia_redshift_wb['sku'] == 'None'),
#                                                'unknown',
#                                                df_mia_redshift_wb['sku'])

# df_mia_redshift_wb['processor'] = np.where((df_mia_redshift_wb['processor'] == 'None'),
#                                                'unknown',
#                                                df_mia_redshift_wb['processor'])

# df_mia_redshift_wb['processor'] = np.where((df_mia_redshift_wb['processor'] == 'None'),
#                                                'unknown',
#                                                df_mia_redshift_wb['processor'])